# Importing OF data and mapping to dolfinx

At first, let us define the variables to read

In [1]:
energy_groups = 6
var_names = ['flux'+str(ii) for ii in range(1, energy_groups+1)]

prec_groups = 8
var_names.extend(['prec'+str(ii) for ii in range(1, prec_groups+1)])

dec_groups = 3
var_names.extend(['dec'+str(ii) for ii in range(1, dec_groups+1)])

var_names.extend(['T', 'p', 'U'])

## Import from OF
Then, let us use *pyforce* to import the data, starting from the mesh.

In [2]:
from pyforce.tools.write_read import import_OF

oF = import_OF('./OpenFOAM_data', extract_dofs=True)
mesh = oF.of_dofs.T

Then, let us focus on the fields

In [3]:
import numpy as np

is_vector = [False] * len(var_names)
is_vector[-1] = True

gdim = mesh.shape[1]

of_snaps = dict()

for field_i, field in enumerate(var_names):
    
    tmp = oF.import_field(field, vector=is_vector[field_i], verbose=True)

    if field_i == 0:
        times = tmp[1]

    if is_vector[field_i]:
        dofs = int(mesh.shape[0]*3)
    else:
        dofs = mesh.shape[0]
    of_snaps[field] = tmp[0]

    del tmp

Importing flux1
Importing flux2
Importing flux3
Importing flux4
Importing flux5
Importing flux6
Importing prec1
Importing prec2
Importing prec3
Importing prec4
Importing prec5
Importing prec6
Importing prec7
Importing prec8
Importing dec1
Importing dec2
Importing dec3
Importing T
Importing p
Importing U


The vector fields are organized as follows.

In [4]:
# u_vec = list()

# for kk in range(gdim):
#     u_vec.append(snaps['U'](0)[kk::3]**2)

# import matplotlib.pyplot as plt
# from matplotlib import cm

# plt.scatter(mesh[:,0], mesh[:,2], c=np.sqrt(sum(u_vec)), cmap = cm.jet)

## Mapping to *dolfinx*

The mesh is generated starting from the `.geo` file.

In [5]:
import gmsh
from mpi4py import MPI
from dolfinx.io import gmshio

mesh_comm = MPI.COMM_WORLD
model_rank = 0
mesh_factor = .011

# Initialize the gmsh module
gmsh.initialize()

# Load the .geo file
gmsh.merge('../EVOL_geom.geo')
gmsh.model.geo.synchronize()

# Set algorithm (adaptive = 1, Frontal-Delaunay = 6)
gmsh.option.setNumber("Mesh.Algorithm", 6)
gmsh.option.setNumber("Mesh.MeshSizeFactor", mesh_factor)
gdim = 2

# Linear Finite Element
gmsh.model.mesh.generate(gdim)
gmsh.model.mesh.optimize("Netgen")

# Import into dolfinx
model_rank = 0
domain, ct, ft = gmshio.model_to_mesh(gmsh.model, MPI.COMM_WORLD, model_rank, gdim = gdim )

domain.topology.create_connectivity(gdim, gdim)
domain.topology.create_connectivity(gdim-1, gdim)

# Finalize the gmsh module
gmsh.finalize()

Info    : Reading '../EVOL_geom.geo'...
Info    : Done reading '../EVOL_geom.geo'
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 40%] Meshing curve 6 (Line)
Info    : [ 50%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 70%] Meshing curve 10 (Line)
Info    : [ 80%] Meshing curve 11 (Line)
Info    : [ 90%] Meshing curve 12 (Line)
Info    : [100%] Meshing curve 13 (Line)
Info    : Done meshing 1D (Wall 0.0405295s, CPU 0.004213s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 1.36024s, CPU 1.29674s)
Info    : 45732 nodes 92055 elements
Info    : Optimizing mesh (Netgen)...
Info    : Done optimizing mesh (Wa

Let us define the functional space for each variable, recalling that the last is for a vector field.

In [6]:
from dolfinx.fem import FunctionSpace
import ufl

fun_spaces = [FunctionSpace(domain, ('Lagrange', 1))]*(len(var_names)-1)
fun_spaces.append(FunctionSpace(domain, ufl.VectorElement("CG", domain.ufl_cell(), 1)))

Let us map the output of OpenFOAM into *dolfinx*

In [7]:
from pyforce.tools.functions_list import FunctionsList
from pyforce.tools.write_read import StoreFunctionsList

snaps = dict() # {var_names[field_i]: FunctionsList(fun_spaces[field_i]) for field_i in range(len(var_names))}

path_snaps = './dolfinx_data/'

for field_i, field in enumerate(var_names):
    
    # Projection in dolfinx
    snaps[field] = oF.foam_to_dolfinx(fun_spaces[field_i], of_snaps[field], variables=['x', 'z'],  cut_value = oF.of_dofs[1,0], verbose='Importing '+field)

    # Store the snapshots
    StoreFunctionsList(domain, snaps[field], field, path_snaps+field, times)

Importing flux1: 500.000 / 500.00 - 0.163 s/it
Importing flux2: 500.000 / 500.00 - 0.162 s/it
Importing flux3: 500.000 / 500.00 - 0.162 s/it
Importing flux4: 500.000 / 500.00 - 0.163 s/it
Importing flux5: 500.000 / 500.00 - 0.163 s/it
Importing flux6: 500.000 / 500.00 - 0.162 s/it
Importing prec1: 500.000 / 500.00 - 0.162 s/it
Importing prec2: 500.000 / 500.00 - 0.162 s/it
Importing prec3: 500.000 / 500.00 - 0.162 s/it
Importing prec4: 500.000 / 500.00 - 0.162 s/it
Importing prec5: 500.000 / 500.00 - 0.162 s/it
Importing prec6: 500.000 / 500.00 - 0.162 s/it
Importing prec7: 500.000 / 500.00 - 0.162 s/it
Importing prec8: 500.000 / 500.00 - 0.162 s/it
Importing dec1: 500.000 / 500.00 - 0.162 s/it
Importing dec2: 500.000 / 500.00 - 0.162 s/it
Importing dec3: 500.000 / 500.00 - 0.162 s/it
Importing T: 500.000 / 500.00 - 0.162 s/it
Importing p: 500.000 / 500.00 - 0.162 s/it
Importing U: 500.000 / 500.00 - 0.317 s/it


Let us store the data using *pickle*

In [8]:
import pickle
pickle.dump([var_names, is_vector, times], open('msfr_uloff.set', 'wb'))